In [0]:
pip install azure

Python interpreter will be restarted.
    ERROR: Command errored out with exit status 1:
     command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-37a9ff03-1279-4d04-9a8d-5e0d2b99e9b1/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-e8b4zqsw/azure_cc0946848b024b1bb34e354ebfee055e/setup.py'"'"'; __file__='"'"'/tmp/pip-install-e8b4zqsw/azure_cc0946848b024b1bb34e354ebfee055e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-48d1k03y
         cwd: /tmp/pip-install-e8b4zqsw/azure_cc0946848b024b1bb34e354ebfee055e/
    Complete output (24 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-e8b4zqsw/azure_cc0946848b024b1bb34e

In [0]:
dbutils.fs.mount(
  source = "wasbs://stagingdata@kunjlearningazure.blob.core.windows.net",
  mount_point = "/mnt/stagingdata",
  extra_configs = {"fs.azure.account.key.kunjlearningazure.blob.core.windows.net":"dunQxSeBrh6vYxOygar9SmA7JyYF15ipC6JWTYKN5Yeps8k5ORBkXecMyARmut1JAkmwWp89AnSe+AStFHPQFA=="})

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-3857937983926771> in <cell line: 1>()
----> 1 dbutils.fs.mount(
      2   source = "wasbs://stagingdata@kunjlearningazure.blob.core.windows.net",
      3   mount_point = "/mnt/stagingdata",
      4   extra_configs = {"fs.azure.account.key.kunjlearningazure.blob.core.windows.net":"dunQxSeBrh6vYxOygar9SmA7JyYF15ipC6JWTYKN5Yeps8k5ORBkXecMyARmut1JAkmwWp89AnSe+AStFHPQFA=="})

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    360                     exc.__context__ = None
    361                     exc.__cause__ = None
--> 362                     raise exc
    363 
    364             return f_with_exception_handling

ExecutionError: An error occurred while calling o381.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /

In [0]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([StructField('datasets', ArrayType(StructType([StructField('customerId', StringType(), True), StructField('orderDate', StringType(), True), StructField('orderDetails', ArrayType(StructType([StructField('productId', StringType(), True), StructField('quantity', LongType(), True), StructField('sequence', LongType(), True), StructField('totalPrice', StructType([StructField('gross', LongType(), True), StructField('net', LongType(), True), StructField('tax', LongType(), True)]), True)]), True), True), StructField('orderId', StringType(), True), StructField('shipmentDetails', StructType([StructField('city', StringType(), True), StructField('country', StringType(), True), StructField('postalCode', StringType(), True), StructField('state', StringType(), True), StructField('street', StringType(), True)]), True)]), True), True), StructField('filename', StringType(), True)])

dfInit = spark.read.option("inferSchema", "true").option("multiline", "true").option("mode", "PERMISSIVE").schema(schema).json("/mnt/stagingdata/bronzeLayer/data1.json")
dfInit.printSchema()
dfInit.show(3)

root
 |-- datasets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- customerId: string (nullable = true)
 |    |    |-- orderDate: string (nullable = true)
 |    |    |-- orderDetails: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- productId: string (nullable = true)
 |    |    |    |    |-- quantity: long (nullable = true)
 |    |    |    |    |-- sequence: long (nullable = true)
 |    |    |    |    |-- totalPrice: struct (nullable = true)
 |    |    |    |    |    |-- gross: long (nullable = true)
 |    |    |    |    |    |-- net: long (nullable = true)
 |    |    |    |    |    |-- tax: long (nullable = true)
 |    |    |-- orderId: string (nullable = true)
 |    |    |-- shipmentDetails: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- postalCode: string (nullable = true)
 |    |    

In [0]:
df = dfInit.withColumn("orderDetail", explode(dfInit.datasets))
df.show(1)

+--------------------+------------+--------------------+
|            datasets|    filename|         orderDetail|
+--------------------+------------+--------------------+
|[{cust5001, 2021-...|orderDetails|{cust5001, 2021-1...|
+--------------------+------------+--------------------+
only showing top 1 row



In [0]:
df = df.withColumn("customerID", col("orderDetail").getItem("customerId"))\
     .withColumn("orderDate", col("orderDetail").getItem("orderDate")).withColumn("orderID", col("orderDetail").getItem("orderId")).drop("datasets")
df.show(1)
df.select(df.orderDetail).show(truncate = False)


+------------+--------------------+----------+--------------------+-------+
|    filename|         orderDetail|customerID|           orderDate|orderID|
+------------+--------------------+----------+--------------------+-------+
|orderDetails|{cust5001, 2021-1...|  cust5001|2021-12-24 00.00....|ord1001|
+------------+--------------------+----------+--------------------+-------+
only showing top 1 row

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|orderDetail                                                                                                                                                   |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, 60}}], ord1001,

In [0]:
df = df.withColumn("shipmentDetailsTrunc", col("orderDetail").getItem("shipmentDetails"))
df.show(1)

df = df.withColumn("country", df.shipmentDetailsTrunc.getItem("country"))\
    .withColumn("postalCode", df.shipmentDetailsTrunc.getItem("postalCode"))\
    .withColumn("state", df.shipmentDetailsTrunc.getItem("state"))\
    .withColumn("street", df.shipmentDetailsTrunc.getItem("street"))\
    .drop("shipmentDetailsTrunc")
df.show()

+------------+--------------------+----------+--------------------+-------+--------------------+
|    filename|         orderDetail|customerID|           orderDate|orderID|shipmentDetailsTrunc|
+------------+--------------------+----------+--------------------+-------+--------------------+
|orderDetails|{cust5001, 2021-1...|  cust5001|2021-12-24 00.00....|ord1001|{Delhi, India, 11...|
+------------+--------------------+----------+--------------------+-------+--------------------+
only showing top 1 row

+------------+--------------------+----------+--------------------+-------+-------+----------+----------+--------+
|    filename|         orderDetail|customerID|           orderDate|orderID|country|postalCode|     state|  street|
+------------+--------------------+----------+--------------------+-------+-------+----------+----------+--------+
|orderDetails|{cust5001, 2021-1...|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040| New Delhi|M.G.Road|
|orderDetails|{cust5002, 2021-1

In [0]:
df = df.withColumn("productDetails", df.orderDetail.getItem("orderDetails"))
df = df.withColumn("productId", df.productDetails.getItem("productId"))\
    .withColumn("quantity", df.productDetails.getItem("quantity"))\
    .withColumn("sequence", df.productDetails.getItem("sequence"))\
    .withColumn("price", df.productDetails.getItem("totalPrice"))\
    .drop("productDetails","orderDetail")
df = df.withColumn("productId",explode("productId"))
df.show(1)

+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+
|    filename|customerID|           orderDate|orderID|country|postalCode|    state|  street|productId|quantity|sequence|               price|
+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|  [2, 3]|  [1, 2]|[{550, 500, 50}, ...|
+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+
only showing top 1 row



In [0]:
df = df.withColumn("quantity",explode("quantity"))\
    .withColumn("sequence",explode("sequence"))
df.show(5)

+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+
|    filename|customerID|           orderDate|orderID|country|postalCode|    state|  street|productId|quantity|sequence|               price|
+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|       2|       1|[{550, 500, 50}, ...|
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|       2|       2|[{550, 500, 50}, ...|
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|       3|       1|[{550, 500, 50}, ...|
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|       3|       2|[{550, 500, 50}, ...|
|order

In [0]:
df = df.withColumn("grossP", df.price.getItem("gross"))\
    .withColumn("netP", df.price.getItem("net"))\
    .withColumn("taxP", df.price.getItem("tax"))
df.show(1)

+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+----------+----------+--------+
|    filename|customerID|           orderDate|orderID|country|postalCode|    state|  street|productId|quantity|sequence|               price|    grossP|      netP|    taxP|
+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+----------+----------+--------+
|orderDetails|  cust5001|2021-12-24 00.00....|ord1001|  India|    110040|New Delhi|M.G.Road|  prd9001|       2|       1|[{550, 500, 50}, ...|[550, 300]|[500, 240]|[50, 60]|
+------------+----------+--------------------+-------+-------+----------+---------+--------+---------+--------+--------+--------------------+----------+----------+--------+
only showing top 1 row



In [0]:
df = df.withColumn("grossP", explode("grossP"))\
    .withColumn("netP", explode("netP"))\
    .withColumn("taxP", explode("taxP"))\
    .drop("price")
# df.show(5)

In [0]:
display(df.limit(10))

filename,customerID,orderDate,orderID,country,postalCode,state,street,productId,quantity,sequence,grossP,netP,taxP
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,500,50
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,500,60
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,240,50
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,240,60
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,300,500,50
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,300,500,60
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,300,240,50
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,300,240,60
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,2,550,500,50
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,2,550,500,60


In [0]:
df = df.withColumn("sr_no", row_number().over(Window.orderBy(monotonically_increasing_id())))

In [0]:
df.limit(5).display()

filename,customerID,orderDate,orderID,country,postalCode,state,street,productId,quantity,sequence,grossP,netP,taxP,sr_no
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,500,50,1
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,500,60,2
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,240,50,3
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,550,240,60,4
orderDetails,cust5001,2021-12-24 00.00.00.000,ord1001,India,110040,New Delhi,M.G.Road,prd9001,2,1,300,500,50,5


In [0]:
from delta import *
def save_to_delta_with_overwrite(resultDf, path, db_name, table_name, mergeCol):
    base_path = path + f"{db_name}/{table_name}"
    if not DeltaTable.isDeltaTable(spark, f"{base_path}"):
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        resultDf.write.mode("overwrite") \
        .format("delta") \
        .option("path", base_path) \
        .save()
    else:
        deltaTable = DeltaTable.forPath(spark, f"{base_path}")
        matchKeys = " AND ".join("old." + col + " = new." + col for col in mergeCol)
        deltaTable.alias("old") \
        .merge(resultDf.alias("new"), matchKeys) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

In [0]:
newDF = df.select("sr_no","orderDate","orderID","quantity","sequence","grossP","netP","taxP")
newDF.limit(5).display()

sr_no,orderDate,orderID,quantity,sequence,grossP,netP,taxP
1,2021-12-24 00.00.00.000,ord1001,2,1,550,500,50
2,2021-12-24 00.00.00.000,ord1001,2,1,550,500,60
3,2021-12-24 00.00.00.000,ord1001,2,1,550,240,50
4,2021-12-24 00.00.00.000,ord1001,2,1,550,240,60
5,2021-12-24 00.00.00.000,ord1001,2,1,300,500,50


In [0]:
df = df.drop("orderDate","orderID","quantity","sequence","grossP","netP","taxP")
df.limit(5).display()

filename,customerID,country,postalCode,state,street,productId,sr_no
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,1
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,2
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,3
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,4
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,5


In [0]:
df.write.mode("overwrite").format("delta").option("mergeSchema", "true").save("dbfs:/mnt/stagingdata/silverLayer/delta")
newDF.write.mode("overwrite").format("delta").option("mergeSchema", "true").save("dbfs:/mnt/stagingdata/silverLayer/delta")

In [0]:
%fs ls /mnt/stagingdata/silverLayer/delta

path,name,size,modificationTime
dbfs:/mnt/stagingdata/silverLayer/delta/_delta_log/,_delta_log/,0,1675160237000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-088a7583-3a17-4e27-8d50-abc411393224-c000.snappy.parquet,part-00000-088a7583-3a17-4e27-8d50-abc411393224-c000.snappy.parquet,4830,1675076502000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-1f7b86d5-6155-4167-abae-1f80b47b7dd1-c000.snappy.parquet,part-00000-1f7b86d5-6155-4167-abae-1f80b47b7dd1-c000.snappy.parquet,3424,1675139880000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-3a8f1db4-e6e2-4661-9f63-a9e9ec998f10-c000.snappy.parquet,part-00000-3a8f1db4-e6e2-4661-9f63-a9e9ec998f10-c000.snappy.parquet,3258,1675139854000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-4d9736ac-0586-407c-83f1-3a4df98d9e70-c000.snappy.parquet,part-00000-4d9736ac-0586-407c-83f1-3a4df98d9e70-c000.snappy.parquet,3258,1675158023000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-669e447a-0155-4ba5-a138-03ca4da4ceec-c000.snappy.parquet,part-00000-669e447a-0155-4ba5-a138-03ca4da4ceec-c000.snappy.parquet,3424,1675160218000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-6f01ecc3-f3c6-4ef2-a567-eacea552b9e0-c000.snappy.parquet,part-00000-6f01ecc3-f3c6-4ef2-a567-eacea552b9e0-c000.snappy.parquet,3424,1675084947000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-7bfb37c5-1b57-4f0e-850e-a68172f3150a-c000.snappy.parquet,part-00000-7bfb37c5-1b57-4f0e-850e-a68172f3150a-c000.snappy.parquet,3258,1675160181000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-90fb4f84-362d-4c37-8172-ff99c5c79061-c000.snappy.parquet,part-00000-90fb4f84-362d-4c37-8172-ff99c5c79061-c000.snappy.parquet,3258,1675085085000
dbfs:/mnt/stagingdata/silverLayer/delta/part-00000-a37e1af4-e137-4865-9428-4d54d36ee5b8-c000.snappy.parquet,part-00000-a37e1af4-e137-4865-9428-4d54d36ee5b8-c000.snappy.parquet,3258,1675084928000


In [0]:
df.display()

filename,customerID,country,postalCode,state,street,productId,sr_no
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,1
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,2
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,3
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,4
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,5
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,6
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,7
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,8
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,9
orderDetails,cust5001,India,110040,New Delhi,M.G.Road,prd9001,10


In [0]:
df.filter(df.state.contains("New Delhi")).withColumn("postalCode", col("postalCode").cast(LongType()))\
  .groupBy("productId").max("postalCode").show()

+---------+---------------+
|productId|max(postalCode)|
+---------+---------------+
|  prd9001|         110040|
|  prd9002|         110040|
+---------+---------------+



In [0]:
save_to_delta_with_overwrite(df, "/mnt/stagingdata/silverLayer/delta", "stagingLayer", "customerData", ["sr_no"])
save_to_delta_with_overwrite(newDF, "/mnt/stagingdata/silverLayer/delta", "stagingLayer", "orderData", ["sr_no"])